In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib as plt

# Riot Developer Portal에서 받은 API KEY입니다.
# 해당 값을 포함한 모든 종류의 KEY는 코드에 직접적으로 노출되지 않도록 하는 것이 좋습니다.
api_key = 'RGAPI-########-####-####-####-############'

# 방금 진행한 내전의 gameId값입니다. 클라이언트 대전기록에서 확인할 수 있습니다.
gameid = 5354085529
requesturl = "https://kr.api.riotgames.com/lol/match/v4/matches/"+str(gameid)+"?api_key="+api_key
# https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/%EC%9C%BC%EB%A5%B8%EA%B3%A0%EC%96%91%EC%9D%B4?api_key=RGAPI-7e3ee2a5-1074-414e-903f-4dd3ffc749a0
r = requests.get(requesturl)

In [2]:
# 불러온 데이터 확인
r.json()

{'status': {'message': 'Forbidden', 'status_code': 403}}

In [ ]:
# 불러온 데이터 중 소환사1의 데이터 확인
r.json()['participants'][0]

In [ ]:
# DataFrame으로 변환해서 확인
pd.DataFrame(r.json()['participants'])

In [ ]:
# nested된 부분 unnest해서 DataFrame으로 변환
df = pd.json_normalize(r.json()['participants'])

In [ ]:
# 확인할 변수
COLUMNS = ["participantId", "teamId", "championId", "spell1Id", "spell2Id", "stats.win",
           "stats.item0","stats.item1","stats.item2","stats.item3","stats.item4","stats.item5","stats.item6",
           "stats.kills","stats.deaths","stats.assists","stats.totalDamageDealt","stats.totalDamageTaken",
           "stats.goldEarned"]

In [ ]:
# 확인할 변수만 가져와서 새로운 DataFrame에 저장
df2 = df[COLUMNS]

In [ ]:
# Column 이름에 불필요한 'stats.'가 붙은 걸 삭제
df2.columns = df2.columns.str.replace('stats.', '', regex = True)

In [ ]:
# 최종 추출 형태 확인
df2

In [ ]:
# 각종 개인 지표 추가해서 새로운 DataFrame에 저장
# KDA = (K + A) / D
# GPM = goldEarned / (gameDuration_In_Munite)
# DPM = damageDealt / (gameDuration_In_Munite)
# DTPM = damageTaken / (gameDuration_In_Munite)
df3 = df2.assign(KDA=lambda x: (x['kills']+x['assists'])/x['deaths'])
df3['GPM'] = df3['goldEarned'] / r.json()['gameDuration'] * 60
df3['DPM'] = df3['totalDamageDealt'] / r.json()['gameDuration'] * 60
df3['DTPM'] = df3['totalDamageTaken'] / r.json()['gameDuration'] * 60

In [ ]:
df3

In [ ]:
# 숫자로 된 각종 Key값에 대한 Value 가져오기
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/11.15.1/data/ko_KR/champion.json")
spell_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/11.15.1/data/ko_KR/summoner.json")
item_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/11.15.1/data/ko_KR/item.json")

In [ ]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]
spell_df = pd.DataFrame(spell_constant.json()['data']).T[['key','name']]
item_df = pd.DataFrame(item_constant.json()['data']).T['name'].rename_axis("key").reset_index()

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])
spell_df['key'] = pd.to_numeric(spell_df['key'])
item_df['key'] = pd.to_numeric(item_df['key'])

In [ ]:
# 매칭되는 곳으로 내용 추가하여 새로운 DataFrame으로 저장
df4 = pd.merge(df3,champion_df,how = 'left', left_on = 'championId', right_on = 'key')

In [ ]:
# DataFrame 확인
df4

In [ ]:
# 중복되는 값 삭제
df4 = df4.rename(columns={'name': 'Champion'}).drop(['championId','key'],axis = 1)

In [ ]:
df4 = pd.merge(df4,spell_df,how = 'left', left_on = 'spell1Id', right_on = 'key')
df4 = df4.rename(columns={'name': 'D'}).drop(['spell1Id','key'],axis = 1)
df4 = pd.merge(df4,spell_df,how = 'left', left_on = 'spell2Id', right_on = 'key')
df4 = df4.rename(columns={'name': 'F'}).drop(['spell2Id','key'],axis = 1)

In [ ]:
df4 = pd.merge(df4,item_df,how = 'left', left_on = 'item0', right_on = 'key')
df4 = df4.rename(columns={'name': 'Item1'}).drop(['item0','key'],axis = 1)
df4 = pd.merge(df4,item_df,how = 'left', left_on = 'item1', right_on = 'key')
df4 = df4.rename(columns={'name': 'Item2'}).drop(['item1','key'],axis = 1)
df4 = pd.merge(df4,item_df,how = 'left', left_on = 'item2', right_on = 'key')
df4 = df4.rename(columns={'name': 'Item3'}).drop(['item2','key'],axis = 1)
df4 = pd.merge(df4,item_df,how = 'left', left_on = 'item3', right_on = 'key')
df4 = df4.rename(columns={'name': 'Item4'}).drop(['item3','key'],axis = 1)
df4 = pd.merge(df4,item_df,how = 'left', left_on = 'item4', right_on = 'key')
df4 = df4.rename(columns={'name': 'Item5'}).drop(['item4','key'],axis = 1)
df4 = pd.merge(df4,item_df,how = 'left', left_on = 'item5', right_on = 'key')
df4 = df4.rename(columns={'name': 'Item6'}).drop(['item5','key'],axis = 1)
df4 = pd.merge(df4,item_df,how = 'left', left_on = 'item6', right_on = 'key')
df4 = df4.rename(columns={'name': 'Trinket'}).drop(['item6','key'],axis = 1)

In [ ]:
# 확인할 변수
COLUMNS_NEW = ["participantId", "teamId","win", "Champion", "D", "F", "KDA",
           "kills","deaths","assists","Item1","Item2","Item3","Item4","Item5","Item6","Trinket","GPM","DPM","DTPM"]

In [ ]:
df5 = df4[COLUMNS_NEW]

In [ ]:
df5